In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
parcel_df = pd.read_csv('Parcel.csv', encoding='latin-1')


In [ ]:
parcel_df= parcel_df.set_index(keys=['Major', 'Minor'])


In [ ]:
# Only take KING COUNTY properties from parcel.csv
parcel_df = parcel_df[parcel_df['DistrictName']=='KING COUNTY']


In [ ]:
parcel_df.head()


In [ ]:
has_water_front = parcel_df['WfntLocation'] > 0
has_water_front.head()

In [ ]:
parcel_df = parcel_df[["SqFtLot", "WfntLocation"]]

In [ ]:
parcel_df.head()

In [ ]:
# wrong DF
unit_df = pd.read_csv('UnitBreakdown.csv')


In [ ]:

unit_df.head()


In [ ]:
# Index unit_df
unit_df = unit_df.set_index(keys=['Major', 'Minor'])


In [ ]:
residentail_blg_df = pd.read_csv('ResBldg.csv')

In [ ]:
residentail_blg_df.columns
residentail_blg_df = residentail_blg_df[['Major', 'Minor', 'Address', 'SqFtTotLiving', 'Bedrooms']]

In [ ]:
sale_df.head()


In [ ]:
sale_parcel_df = pd.merge(sale_df, parcel_df, right_index=True, left_on=['Major', 'Minor'])


In [ ]:
all_df = pd.merge(sale_parcel_df, unit_df, right_index=True, left_on=['Major', 'Minor'])

In [ ]:
unit_df.shape


In [ ]:
sale_df = pd.read_csv('Sale.csv')


In [ ]:
sale_df.columns
# removing the 2 rows without Major+Minor index


In [ ]:
sale_df['Major'] = pd.to_numeric(sale_df['Major'], errors='coerce')
sale_df['Minor'] = pd.to_numeric(sale_df['Minor'], errors='coerce')


In [ ]:
sale_df = sale_df.dropna()


In [ ]:
sale_df.isna().sum()


In [ ]:
sale_df = sale_df.set_index(keys=['ExciseTaxNbr','Major', 'Minor'])

sale_df.head()


In [ ]:
parcel_df.head()


In [ ]:
sale_df.isna().sum()


In [ ]:
sale_df.info()


In [ ]:
sale_df.head(1)


In [ ]:
parcel_df


In [ ]:
unit_df.info()



In [ ]:
unit_df.head()

In [ ]:
unit_df['NbrBedrooms'].astype(int)

In [ ]:
#assuming S means Studio
nbr = unit_df['NbrBedrooms']


In [ ]:
nbr = nbr.str.replace('S', '0')


In [ ]:
#Replace spaces with 0
nbr = nbr.str.replace(' ', '0')


In [ ]:
nbr = nbr.astype(float)


In [ ]:
unit_df['NbrBedrooms'] = nbr


In [ ]:
unit_df.info()


In [ ]:
# what im going to do
wftn = parcel_df['WfntLocation']


In [ ]:
wftn = wftn.astype(bool)


In [ ]:
parcel_df['WfntLocation'] =wftn


In [ ]:
parcel_df.info()


In [ ]:
parcel_df.head(5)



In [ ]:
unit_df.head(5)



In [ ]:
parcel_unit_df = pd.merge(parcel_df, unit_df, how='inner', left_index=True, right_index=True)

In [ ]:
parcel_unit_df.info()


In [ ]:
sale_df.head()
sale_df.reset_index(level=0,inplace=True)

In [ ]:
sale_df


In [ ]:
unit_price_df = pd.merge(sale_df, unit_df, how='inner', left_index=True , right_index=True)

In [ ]:
# YAY! the inner join worked
unit_price_df

In [ ]:
# this doesnt work
housing_df = pd.merge(unit_price_df, parcel_df, how='left', left_index=True , right_index=True)

In [ ]:
housing_df.info()


In [ ]:
sale_df = sale_df[['DocumentDate','SalePrice', 'PropertyClass']]


In [ ]:
sale_df


In [ ]:
new_df = pd.merge(sale_df, residentail_blg_df, left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

In [ ]:
new_df.head()


In [ ]:
parcel_res_df =pd.merge(parcel_df, residentail_blg_df, left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

In [ ]:
parcel_res_df.shape

In [ ]:
housing_df = pd.merge(parcel_df, new_df, left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

In [ ]:
housing_df.shape

In [ ]:
housing_df.columns